In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
import lightgbm
import catboost
import xgboost
import shap
#import seaborn as sns
from catboost import *
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, auc
#%matplotlib
###########loading data##########
fdata=pd.read_csv('database_filled_ST.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                      'Operating Temperature (℃)',#0
                      'Flow Rate (mL min-1)',#1
                      'Active Area (cm2)', #2
                      'Ir wt. %',#3
                      'Ru wt.%',#4
                      'O wt. %',#5
                      'I/C in Anode',#6
                      'Pt wt. %',#7
                      'I/C in Cathode',#8 
                      'Anode Precious Metal Loading (mg cm-2 Ir/Ru/Pt/Pd)',#9
                      'Cathode Precious Metal Loading (mg cm-2 Pt/Pd)',#10
                      'Membrane Thickness (μm)',#11
                      'EW',#12
                      'Maximum Stability Current Density (A cm-2)',#13
                      'Stability Test Time (h-1)'#14
                        ]]
###########train test splitting##########
raw_param=raw_data.iloc[:,0:15]
print('ready')


In [ ]:
ST10=fdata.loc[:,['CL_10']]

In [ ]:
def summarize(list1):
    total=0
    for ele in range(0, len(list1)):
        total = total + list1[ele]
    return total

In [ ]:
def gridsearch(model,param,algorithm_name):
    print('start')
    grid = GridSearchCV(model,param_grid=param,cv=5,n_jobs=-1)
    grid.fit(X_train,y_train)
    print('Best Classifier:',grid.best_params_,'Best Score:', grid.best_score_)
    best_model=grid.best_estimator_
    prediction_train=best_model.predict(X_train)
    prediction_test=best_model.predict(X_test)
    final_result=classification_report(y_test,prediction_test,output_dict=True)
    ##############################################################
    y_score=best_model.predict_proba(X_test)
    y_score=y_score[:,1]
    auc_curve(y_test,y_score,algorithm_name)
    ##############################################################
    print(classification_report(y_train,prediction_train))
    print(classification_report(y_test,prediction_test))
    print(final_result['accuracy'])
    ###########generating a figure##########
    print(algorithm_name)
    print(best_model.feature_importances_)
    return best_model

In [ ]:
def auc_curve(y_label, y_pre,algorithm_name):
    y_label = y_label + 1
    y_pre = y_pre + 1
    fpr, tpr, thersholds = roc_curve(y_label, y_pre, pos_label=2)
    roc_auc = auc(fpr, tpr)
    x_line=np.arange(0,1.01,0.01)
    y_line=np.arange(0,1.01,0.01)
    print('auc',roc_auc)
    fig=plt.figure()
    plt.plot(fpr, tpr, 'k--', label='ROC (AUC/area = {0:.2f})'.format(roc_auc), lw=2)
    plt.plot(x_line,y_line,c='red')
    plt.xlim([-0.05, 1.05])  # 设置x、y轴的上下限，以免和边缘重合，更好的观察图像的整体
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')  # 可以使用中文，但需要导入一些库即字体
    plt.title('ROC Curve of %s' %algorithm_name)
    plt.legend(loc="lower right")
    plt.savefig('ROC Curve of %s ST10.png' %algorithm_name)

In [ ]:
seed=1743
X_train, X_test, y_train, y_test = train_test_split(raw_param, ST10, test_size=.15,random_state=seed)

In [ ]:
def shap_plot_interaction(best_model,algorithm_name,interacted_features):
    print(algorithm_name)
    SHAP_INPUT=raw_param
    SHAP_OUTPUT=ST10
    print('train finished')
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)

    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        shap_values=shap_values[:,:-1]
#         print(shap_values)
        interaction_values = best_model.get_feature_importance(Pool(SHAP_INPUT),type=EFstrType.ShapInteractionValues)
        interaction_values=interaction_values[:,:-1,:-1]
        shap.dependence_plot(interacted_features[0], shap_values, SHAP_INPUT,interaction_index= interacted_features[1])
        shap.dependence_plot(interacted_features[1], shap_values, SHAP_INPUT,interaction_index= interacted_features[0])
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,SHAP_INPUT)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)        
        interaction_values = shap.TreeExplainer(best_model).shap_interaction_values(SHAP_INPUT) 
#         print(shap_values)
        shap.dependence_plot(interacted_features[0], shap_values[1], SHAP_INPUT,interaction_index= interacted_features[1])
        shap.dependence_plot(interacted_features[1], shap_values[1], SHAP_INPUT,interaction_index= interacted_features[0])
    else:
        explainer = shap.TreeExplainer(best_model,SHAP_INPUT)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        interaction_values = shap.TreeExplainer(best_model).shap_interaction_values(SHAP_INPUT)
#         print(shap_values)
        shap.dependence_plot(interacted_features[0], shap_values, SHAP_INPUT,interaction_index= interacted_features[1])
        shap.dependence_plot(interacted_features[1], shap_values, SHAP_INPUT,interaction_index= interacted_features[0])
from pdpbox import pdp
def plot_pdp_interact_ANN(model, df, f_list, cluster_flag=False, nb_clusters=None, lines_flag=False):
    
    # Create the data that we will plot
    inter1 = pdp.pdp_interact(model, df, model_features=df.columns.tolist(), features=f_list,num_grid_points=[20,20])
    # plot it
    settings = {
            'contour_color':  'white',
            'font_family': 'Arial',
            # matplotlib color map for interact plot
            'cmap': 'viridis',
            # fill alpha for interact plot
            'inter_fill_alpha': 0.8,
            # fontsize for interact plot text
            'inter_fontsize': 7,
        }
    pdp.pdp_interact_plot(
    pdp_interact_out=inter1, feature_names=f_list, plot_type='contour',figsize=(10,10),x_quantile=True, plot_pdp=True,plot_params=settings
)
from sklearn.inspection import plot_partial_dependence
from sklearn.utils import validation
def pdp_plot_2d(best_model,f_list):
    print('start')
    validation.check_is_fitted(estimator=best_model)
    my_plots =plot_partial_dependence(best_model, features=[f_list], X=raw_param, percentiles=(0, 1),grid_resolution=100)
def pdp_plot_2d_XG_CAT(best_model,f_list):
    print('start')
    best_model.dummy_ = "dummy"
    validation.check_is_fitted(estimator=best_model)
    my_plots =plot_partial_dependence(best_model, features=[f_list], X=raw_param, percentiles=(0, 1),grid_resolution=100)

In [ ]:
###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestClassifier = ensemble.RandomForestClassifier(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
'n_estimators':[50],
'max_depth':[5],
'criterion':['gini'],
'max_features':['auto']
}
RF=gridsearch(model_RandomForestClassifier,param_RF,'Random Forest')

In [ ]:
%matplotlib
inter_feature_list=[['Ir wt. %','Ru wt.%'],["Cathode Precious Metal Loading (mg cm-2 Pt/Pd)","I/C in Cathode"],['Anode Precious Metal Loading (mg cm-2 Ir/Ru/Pt/Pd)','I/C in Anode'],['Membrane Thickness (μm)','EW'],['Stability Test Time (h-1)','Maximum Stability Current Density (A cm-2)']]

In [ ]:
for inter_feature in inter_feature_list:
    shap_plot_interaction(RF,algorithm_name="Random Forest",interacted_features=inter_feature)

In [ ]:
for inter_feature in inter_feature_list:
    pdp_plot_2d(RF,inter_feature)